# Import MIS Data 

## Setup and Helper Functions

In [ ]:
import re
import json

from pathlib import Path

import pandas as pd
import numpy as np
from scipy import signal

from fau_colors import cmaps
import biopsykit as bp
from empkins_io.sensors.mis import MisDataset

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
plt.close("all")

palette = sns.color_palette(cmaps.faculties)
sns.set_theme(context="notebook", style="ticks", font="sans-serif", palette=palette)

plt.rcParams["figure.figsize"] = (8, 5)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"

palette

In [ ]:
data_path = sorted(Path("data").glob("*.mat"))[0]
data_path

## Load Data

In [ ]:
dataset = MisDataset.from_mat_file(data_path)

## Convert to DataFrame

### Configure Index

In [ ]:
# index: sample counter
data = dataset.data_as_df()
data.head()

In [ ]:
# index: time in seconds since recording start
data = dataset.data_as_df(index="time")
data.head()

In [ ]:
# index: UTC time
data = dataset.data_as_df(index="utc_datetime")
data.head()

In [ ]:
# index: local time
data = dataset.data_as_df(index="local_datetime")
data.head()

## Extract Data

### Heart Rate

In [ ]:
dataset.heart_rate(index="local_datetime")

### Respiration

In [ ]:
dataset.respiration(fs_out=100, index="local_datetime").plot()

## Select Specific Time Interval

### Time Stamps

In [ ]:
data = dataset.data_as_df(index="local_datetime")

ts1 = data.index[34]
ts2 = data.index[10455]
print(ts1)
print(ts2)

In [ ]:
data.loc[ts1:ts2]

### Absolute Time

### Relative Time

In [ ]:
data.first("3s")